## The Tensor Network Data Structure Used By PicoQuant

In this notebook we provide some details about how to interact with tensor networks in PicoQuant and provide some details about the data structures representing them. Briefly, the data structure that PicoQuant uses to represent a tensor network consists of three julia structs: 

1. A Node struct representing a tensor in the tensor network.
2. An Edge struct representing an edge in the tensor network graph.
3. A TensorNetworkCircuit struct to hold references to all nodes and edges in the tensor network.

We will first create a tensor network and then describe how the user can interact with it. For the interested reader, we provide further details on the composition of the julia structs used in a tensor network at the end of the notebook.

Below, we create a tensor network for the same GHZ quantum circuit from the introduction notebook.

In [ ]:
using PicoQuant

# Create an empty tensor network for a circuit with 3 qubits
InteractiveBackend()
tn = TensorNetworkCircuit(3)

# Create tensors for the hadamard and CNOT gates
hadamard_data = gate_tensor(:H)
CNOT_data = gate_tensor(:CX)

# Add the gates to the circuit in the desired order.
add_gate!(tn, hadamard_data, [1])
add_gate!(tn, CNOT_data, [2, 1])
add_gate!(tn, CNOT_data, [3, 2])

# Add nodes to the network with the state of the input qubits.
add_input!(tn, "000")

# Plot a graph of the tensor network.
plot(tn, showlabels=true)

As described in the introduction notebook, and illustrated above, adding nodes to a network can be done using the `add_input!`, `add_output!` and `add_gate!` functions.

#### Accessing nodes and edges of a tensor network

Every node added to the network will be assigned a unique symbol which can be used to identify it as shown in the plot above. Both the symbol and node will be stored as a key-value pair in a dictionary contained in the TensorNetworkCircuit. Namely, the 'i'-th node to be added to the network will be assigned the julia symbol `:node_i`. If we need to inspect the contents of a particular node, we can use this symbol to retrieve the node struct from the dictionary it is stored in as follows.

In [ ]:
tn.nodes[:node_2]

Note, the node struct contains an array of indices for the corresponding tensor and does not store the tensor data. The associated tensor data are stored by the backend under the same node label. If we are using an interactive backend, which stores the tensor data in a dictionary similar to the nodes dictionary, we can access the tensor data as shown below.

In [ ]:
backend.tensors[:node_1]

Similarly, edges are also assigned julia symbols of the form `:index_i`, when they are created. The edge structs are stored with their assigned symbols as key-value pairs in a dictionary within the tensor network, just like the nodes dictionary:

In [ ]:
tn.edges[:index_2]

#### Getting the neighbours of a node within a tensor network
For some algorithms, it can be useful to have quick access to neighbouring nodes of a particular node of interest. To this end, PicoQuant provides four functions that provide just that. The `neighbours(tn, :node_i)` function will return an array of node labels for the nodes in the network `tn` that are connected to the node labeled by `:node_i`.

In [ ]:
neighbours(tn, :node_2)

The graph of a quantum circuit is a directed graph where edges have a natural direction associated with them running from input to output. The functions `inneighbours` and `outneighbours` will return similar arrays to the one returned by `neighbours` but containing the nodes connected to `:node_i` via incoming and outgoing edges respectively.

In [ ]:
node = :node_2
println("The incoming neighbours of $node are: ", inneighbours(tn, node))
println("The outgoing neighbours of $node are: ", outneighbours(tn, node))

Similarly, we can access the incoming and outgoing edges of a node using the following functions

In [ ]:
println("The incoming edges of $node are: ", inedges(tn, node))
println("The outgoing edges of $node are: ", outedges(tn, node))

When a node is decomposed into multiple nodes, as described in notebook 4, the edges connecting them do not have a natrual direction associated with them and are called virtual edges. For networks with virtual edges, there is a `virtualneighbours` function which behaves just like the `neighbours` function but only returns neighbours connected by virtual edges.

#### Getting the number of nodes or edges in a tensor network

At any point, the total number of nodes and edges that are currently in the tensor network can be retrieved from a dictionary of counters contained in the TensorNetworkCircuit structure as shown below. These counters are used and incremented internally to generate labels for newly created nodes and edges. 

In [ ]:
println("The number of nodes in the tensor network is: ", tn.counters["node"])
println("The number of edges in the tensor network is: ", tn.counters["index"])

### The tensor network data structure

We now provide some details of the underlying data structure for the interested reader. As mentioned at the start of this notebook, PicoQuant uses three julia structs to represent a tensor network: `TensorNetworkCircuit`, `Node` and `Edge`

#### TensorNetworkCircuit struct

The TensorNetworkCircuit struct plays the role of a parent struct for the nodes and edges in a network. Namely, it holds references to all the Node and Edge instances representing a particular tensor network. This makes it convenient to access and manipulate nodes/edges. The data fields of the TensorNetworkCircuit structure as listed below:

1. An integer for the number of qubits in the circuit being represented.
2. An array of index labels connected to the input qubits.
3. An array of index labels connected to the output qubits.
4. An ordered dictionary containing the nodes of the tensor network.
5. An ordered dictionary containing the edges of the tensor network.
6. A dictionary of counters keeping track of how many nodes and edges are in the tensor network.

When a node or edge is added to the tensor network, the TensorNetworkCircuit instance is given the corresponding Node/Edge struct and a unique label (saved as a julia symbol) to be used as an identifier for the struct. The pair are save in the dictionary mentioned above with the unique label as the key and the struct as the value.

#### Node struct
The main purpose of the Node struct is to hold an array of index labels (saved as julia symbols) corresponding to the indices of the tensor the node represents. These index labels are used to determine how tensors should be reshaped, permuted and contracted whenever the tensor network is altered. The data fields of the node struct are:

1. An array of julia symbols representing tensor indices
2. The Julia symbol under which the backend has saved the corresponding tensor data.

Note, tensor data is not stored in the Node struct or in the TensorNetworkCircuit struct. Instead, tensor data is stored by the backend the user initialises PicoQuant with before any tensors are created. When the user adds a gate to the network, a symbol is created as a label for the gate data.

Below, we get the instance of Node corrsponding to the first tensor added to the network and show how we typically access the indices of a node. We can compare the printed indices with those displayed in the above graph of the tensor network and see that they match as expected.

In [ ]:
node = tn.nodes[:node_1]
println("The indices contained in node 1 are: ", node.indices)

#### Edge struct

Edge structs are used to keep track of which nodes are connected by common/contracted indices. When a node is added to the tensor network, an edge struct is created for each index of the node (if one doesn't already exist) and is stored with the corresponding index label in the edges dictionary mentioned at the beginning of this notebook. Each edge struct will be given two node labels to store that identify the two nodes in the network which share the corrsponding index. If the index associated with an edge is an open index, then one of the node labels in the edge struct is replaced with the constant `nothing`.

If none of the gates added to the circuit are decomposed into tensors connected by virtual bonds, the tensor network natrually forms a directed graph with edges originating from the input qubits and moving towards the output qubits. As such, the node labels stored in an Edge are saved under the variable names src (source) and dst (destination).

In [ ]:
first_edge = tn.edges[:index_1]
println("The source node of the edge labeled :index_1 is: ", first_edge.src)
println("The destination node of the edge labeled :index_1 is: ", first_edge.dst)